# Create interactive plots

This notebook is to assist in exploring the results

In [ ]:
%matplotlib inline

import sys
import warnings
import odc.geo.xr
import xarray as xr
import branca.colormap as cm
import matplotlib as mpl
from odc.geo.xr import assign_crs

sys.path.append('/g/data/os22/chad_tmp/AusEFlux/src/')
from _utils import round_coords

## Analysis Parameters


In [ ]:
lin_or_circ = 'circular'

p_average = assign_crs(xr.open_dataset(f'/g/data/os22/chad_tmp/Aus_phenology/results/mean_phenology_perpixel_{lin_or_circ}.nc'), crs='EPSG:4326')
p_trends = assign_crs(xr.open_dataset(f'/g/data/os22/chad_tmp/Aus_phenology/results/trends_phenology_perpixel_{lin_or_circ}_TEST.nc'), crs='EPSG:4326')
# p_parcorr = assign_crs(xr.open_dataset(f'/g/data/os22/chad_tmp/Aus_phenology/results/ios_analysis_perpixel_{lin_or_circ}.nc'), crs='EPSG:4326')

## Masking

In [ ]:
crops = xr.open_dataset('/g/data/os22/chad_tmp/Aus_phenology/data/croplands_5km.nc')['croplands']
crops = xr.where(crops!=1, 0, 1) #all cropping
crops = round_coords(crops)

In [ ]:
# p_trends = round_coords(p_trends)
# p_trends = p_trends.where(crops)

# p_parcorr = round_coords(p_parcorr)
# p_parcorr = p_parcorr.where(crops)

## Long-term average phenology




In [ ]:
var = 'SOS'
cmap='twilight'
vmin, vmax= 0, 365
m = p_average[var].odc.explore(
            tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
            attr = 'Esri',
            name = 'Esri Satellite',
            cmap=cmap,
            vmin=vmin,
            vmax=vmax
)

cmaps = mpl.colormaps[cmap]
colormap = cm.LinearColormap(cmaps.colors, vmin=vmin,vmax=vmax, caption=var)
m.add_child(colormap)

## Trends

In [ ]:
var = 'LOS'
cmap='PuOr'
vmin, vmax= -2,2

m = p_trends[var+'_slope'].odc.explore( #.where(p_trends[var+'_p_value']<=0.05)
            tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
            attr = 'Esri',
            name = 'Esri Satellite',
            cmap=cmap,
            vmin=vmin,
            vmax=vmax
)
m

In [ ]:
var = 'SOS'
cmap='RdBu_r'
vmin, vmax= -2,2

m = p_trends[var+'_slope'].odc.explore( #.where(p_trends[var+'_p_value']<=0.05)
            tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
            attr = 'Esri',
            name = 'Esri Satellite',
            cmap=cmap,
            vmin=vmin,
            vmax=vmax
)
m

In [ ]:
var = 'EOS'
cmap='RdBu_r'
vmin, vmax= -2,2

m = p_trends[var+'_slope'].odc.explore( #.where(p_trends[var+'_p_value']<=0.05)
            tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
            attr = 'Esri',
            name = 'Esri Satellite',
            cmap=cmap,
            vmin=vmin,
            vmax=vmax
)
m
# cmaps = mpl.colormaps[cmap]
# colormap = cm.LinearColormap(cmaps, vmin=vmin, vmax=vmax, caption=var)
# m.add_child(colormap)

In [ ]:
var = 'IOS'
cmap='PuOr'
vmin, vmax= -1,1

m = p_trends[var+'_slope'].where(p_trends[var+'_p_value']<=0.05).odc.explore(
            tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
            attr = 'Esri',
            name = 'Esri Satellite',
            cmap=cmap,
            vmin=vmin,
            vmax=vmax
)
m

In [ ]:
season_per_year = p_average['n_seasons']/p_average['n_years']
non_seasonal = xr.where((season_per_year <= 0.90),1,0)
extra_seasonal = xr.where((season_per_year >= 1.1),1,0)
seasonality_mask = (non_seasonal | extra_seasonal)

In [ ]:
vmin,vmax=0.7,1.3
cmap='RdYlBu'

m = season_per_year.odc.explore(
            tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
            attr = 'Esri',
            name = 'Esri Satellite',
            cmap=cmap,
            vmin=vmin,
            vmax=vmax
)
m

## Percentage of pixels with change

In [ ]:
import numpy as np

In [ ]:
for var in ['SOS', 'POS', 'EOS', 'LOS']:
    trends = p_trends[var+'_slope'].where(seasonality_mask!=1)
    pval = p_trends[var+'_p_value'].where(seasonality_mask!=1)
    
    total_nonnan = (~np.isnan(trends)).sum().values.item()
    total_significant = (~np.isnan(trends.where(pval<=0.05))).sum()
    significant_positive = (~np.isnan(trends.where(trends>0).where(pval <=0.05))).sum().values.item()
    significant_negative = (~np.isnan(trends.where(trends<0).where(pval <=0.05))).sum().values.item()
    
    total_percent_changed = total_significant / total_nonnan * 100
    total_percent_positive_change = significant_positive / total_nonnan * 100
    total_percent_negative_change = significant_negative / total_nonnan * 100
    
    print(f'{total_percent_changed:.2f} % of valid pixels in Australia has seen significant change in {var}')
    print(f'{total_percent_positive_change:.2f} % of valid pixels in Australia has seen significant positive trends in {var}')
    print(f'{total_percent_negative_change:.2f} % of valid pixels in Australia has seen significant negative trends in {var}')
    print('\n')


In [ ]:
p_trends[var+'_slope'].plot(vmin=-2, vmax=2)

In [ ]:
for var in ['vSOS', 'vPOS', 'vEOS', 'AOS', 'ROG', 'ROS', 'IOS']:
    trends = p_trends[var+'_slope'].where(seasonality_mask!=1)
    pval = p_trends[var+'_p_value'].where(seasonality_mask!=1)
    
    total_nonnan = (~np.isnan(trends)).sum().values.item()
    total_significant = (~np.isnan(trends.where(pval<=0.05))).sum()
    significant_positive = (~np.isnan(trends.where(trends>0).where(pval <=0.05))).sum().values.item()
    significant_negative = (~np.isnan(trends.where(trends<0).where(pval <=0.05))).sum().values.item()
    
    total_percent_changed = total_significant / total_nonnan * 100
    total_percent_positive_change = significant_positive / total_nonnan * 100
    total_percent_negative_change = significant_negative / total_nonnan * 100
    
    print(f'{total_percent_changed:.2f} % of valid pixels in Australia have seen significant change in {var}')
    print(f'{total_percent_positive_change:.2f} % of valid pixels in Australia have seen significant positive trends in {var}')
    print(f'{total_percent_negative_change:.2f} % of valid pixels in Australia have seen significant negative trends in {var}')
    print('\n')

## Seasonal component most correlated with IOS

In [ ]:
import numpy as np
import sys
sys.path.append('/g/data/os22/chad_tmp/AusEFlux/src/')
from _prediction import allNaN_arg

import distinctipy
from matplotlib.colors import LinearSegmentedColormap

In [ ]:
max_corr = allNaN_arg(np.abs(p_parcorr.to_array()), dim='variable',stat='max', idx=False)

In [ ]:
categories = list(p_parcorr.data_vars)
colours = distinctipy.get_colors(len(categories), pastel_factor=0.5, rng=2)
cmap = LinearSegmentedColormap.from_list("cwc", colours, N=len(categories))
vmin, vmax= 0,7

m = max_corr.odc.explore(
            tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
            attr = 'Esri',
            name = 'Esri Satellite',
            cmap=cmap,
            vmin=vmin,
            vmax=vmax
)
m